# Lab Assignment One: Exploring Table Data

### Luis Garduno

## 1. Business Understanding

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; About League of Legends :
Developed by Riot Studios, League of Legends, or "LoL", is an online multiplayer
video-game that is available to Windows/MacOS users. LoL consists 2 teams ('Blue
& 'Red') facing each other, where the main objective is to destroy the opposing
teams 'Nexus', or home base, while facing obstacles like destroying damage dealing
towers & eliminating players throughout the way. Perks & gold are able to be
obtained by players/teams through completing tasks such as eliminating players,
enemy creeps, or dragons. Players then spend the gold to purchase items that help
raise the power of their abilities.

League of Legends offers different game modes, such as ranked. In this game mode,
players are given a rank based off of the number of wins + the number of games
played. "Diamond" is one of the highest ranks a player may obtain and is known
to be extremely competitive. A ranked game on average lasts 30-45 minutes. The
dataset we will be using contains the first 10 minute analytics of each team
for different diamond ranked matches.

Once the data is analyzed, teams/players would be able to conceptualize the level
of priority different attributes have during early stages of diamond ranked
matches. They could then use this information to adjust their strategy to one
proven to win matches.

-------------------------------------

Dataset : https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min

Question Of Interest : As of the first 10 minutes, which team will win?

## 2. Data Understanding

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.1 Data Description

In [ ]:
import numpy as np
import pandas as pd

# Load in the dataset into a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/luisegarduno/MachineLearning_Projects/master/Datasets/high_diamond_ranked_10min.csv')

df.info()

---------------------------------

Printing out the information about the dataframe we able to see that there are a
total of 9,879 instances, and 39 attributes.

Additionally we are able to see that there are 19 of the same attributes for each
the blue & red team (columns 1-19 are the same as 20-38).

Attributes for each team includes :
- Wards placed & destroyed
- Total number of kills, deaths, & assists
- First Bloods (1st elimination of the game)
- Total : towers destroyed, gold, experience
- Average : level, CS per minute, & gold per minute
- Difference in gold & experience between the teams
- Objective eliminations : elite monsters, dragons, heralds, minions, & jungle minions

Attributes such as total gold, experience, objectives eliminations, towers destroyed, etc.
will be of type integer (int64) because they will always be whole numbers. Attributes involving
averages such as cs per minute, gold per minute, & level, should be the only of double-precision floating-point
format (float64).

The data type for "blueWins" and "first bloods" could be changed to be of type boolean, but because we are wanting to
visualize these attributes, optimally it is best to keep these as integer data types. As a result,
the data types presented for each attribute are correct and should not be changed.

Below is a brief description of some of the key attributes.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# describe dataframe
df.describe()

In [ ]:
df_data_description = pd.DataFrame({'Features' : ['blueWins','WardsPlaced / WardsDestroyed','FirstBlood','Kills / Deaths / Assists','TowersDestroyed',
                                                  'TotalGold','AvgLevel','TotalExperience','CSPerMin','GoldPerMin']})

df_data_description['Description'] = ['whether blue team won or not','number of total wards placed or destroyed by team','team with the first kill of game',
                                      'total number of kills, deaths, or assists of team','total number of towers destroyed by team','total gold obtained by team',
                                      'average level of all players on team','total experience points accumulated by team','average creep score per minute',
                                      'average gold obtained per minute']

df_data_description['Feature type'] = ['Discrete','Continuous','Discrete','Continuous','Continuous','Continuous','Continuous','Continuous','Continuous','Continuous']

df_data_description['Attribute Type'] = ['nominal','ratio','nominal','ratio','ratio','ratio','ratio','ratio','ratio','ratio']

df_data_description['Range'] = ['0: red team won; 1: blue team won','placed: 5 - 250;destroyed: 0 - 27','0: did not get first kill; 1: team obtained first kill',
                                'kills: 0-22;deaths: 0-22;assists: 0-29','0 - 2','11,000 - 25,000','4.5 - 8.5','10,000 - 24,000','10.0 - 30.0','1,100.0 - 2,000.0']
df_data_description

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2 Data Quality

Using the `missingno` package, we are able to additionally confirm that all the data is complete
and there is no missing entries with the dataset. If there was missing data, we could impute the
missing values by using the k-nearest neighbor. But if an instance was missing a majority of its
attributes, it would be removed from the dataset.

The number of unique values in the column "gameId" is printed to verify that all instances
are weighted equally.

In [ ]:
import missingno as mn

mn.matrix(df)

# Count unique values in column 'gameId' of the dataframe
print('Number of unique values in column "gameId" : ', df['gameId'].nunique())

dup_df = df.duplicated()
print('Duplicates : ', len(df[dup_df]))

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3 Cleaning the Dataset

After confirming there are no duplicates in the data, the "gameId" column can be removed since it
will have no impact on the results.

Using the correlation feature from the `pandas` package, for each team we find the names of
attributes that correlate most with winning (correlation >= 7%). The names of these attributes
are stored in a array for later use.

In [ ]:
del df['gameId']

red_col = df.corr()[df.corr()['blueWins'] <= -0.07].index.values
blue_col = df.corr()[df.corr()['blueWins'] >= 0.07].index.values

----------------------

## 3. Data Visualization

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.1 Data Exploration

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.1.1 Win rate by Team
Does being on a specific team have an impact on winning?

For context :
- Red team defends the top right Nexus
- Blue team defends the bottom left Nexus

In [ ]:
ax = sns.countplot(x="blueWins", data=df, palette=['red', 'blue'])
ax.set_title('Win Rate by Team')
ax.set_xlabel('Teams')
ax.set_xticks([0,1])
ax.set_xticklabels(['Red', 'Blue'])
ax.set_ylabel('Frequency')

Because each team puts you on a specific side of the map, some may think that having a specific perspective on the map
may make an impact on your chances of winning. As shown above, the win rate for each team is symmetrical within the
large dataset, meaning that defending a specific side of the map will likely not have an impact on the game outcome.


#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.1.2 Win rate by kill/death/assist ratio

In [ ]:
# Formula to calculate KDA ratio ------> KDA = (Kills + Assists) / Deaths
df['redKDA'] = np.int64((df['redKills'] + df['redAssists']) / df['redDeaths'])
df['blueKDA'] = np.int64((df['blueKills'] + df['blueAssists']) / df['blueDeaths'])
df_win = df[df["blueWins"]==1]
df_lose = df[df["blueWins"]==0]



#df_blueKDA = df.groupby(by=['blueKDA', 'blueWins'])
#win_rate_b = (df_blueKDA.blueWins.sum() / df.blueWins.count() * 100)
#win_rate_b = win_rate_b.drop(index=win_rate_b.index[[0]])
#
#df_redKDA = df.groupby(by=['redKDA', 'blueWins'])
#win_rate_r = (df_redKDA.blueWins.sum() / df.blueWins.count() * 100)
#win_rate_r = win_rate_r.drop(index=win_rate_r.index[[0]])
##df_blueKDA.plot.kde(bw_method=0.1,secondary_y=True)
#df_blueKDA.max()

#print("Largest Blue KDA (Win) : ", df_win['blueKDA'].max())
#print("Largest Red KDA (Loss) : ", df_win['redKDA'].max())
#print("Largest Blue KDA (Loss) : ", df_lose['blueKDA'].max())
#print("Largest Red KDA (Win) : ", df_lose['redKDA'].max())
#
#ax_blue = win_rate_b.plot(label='BlueTeam KDA',color='blue')
#ax_red = win_rate_r.plot(label='RedTeam KDA',color='red')
#ax_blue.set_xlabel('KDA Ratio')
#ax_blue.set_ylabel('Frequency')
#ax_blue.set_title('Win % based on blueKDA')
#ax_blue.legend(loc='upper right')
#plt.show()

jitter_values = ['blueKills','blueAssists','blueDeaths','blueWins']

df_jitter = df[jitter_values].copy()
df_jitter[['blueKills','blueAssists','blueDeaths']] += np.random.rand(len(df_jitter),3)/2
sns.pairplot(df_jitter,hue="blueWins",height=2,plot_kws=dict(s=20,alpha=0.15,linewidth=0),palette=['red','blue'])

plt.show()

A KDA, or Kills/Deaths/Assists, ratio is highly significant when determining whether a team will win or not.
A KDA ratio is calculated using the formula shown in the code (Kills + Assists / Deaths).

We additionally use a jitter plot to demonstrate KDA. The number of deaths as shown display the correlation
between the number of deaths a team has and losing.

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.1.3 Win rate by objective eliminations

In [ ]:
df_blue_obj = df.groupby(by=['blueEliteMonsters', 'blueDragons', 'blueHeralds'])
obj_blue = pd.crosstab([df['blueEliteMonsters'],df['blueDragons'],df['blueHeralds']],df.blueWins.astype(bool))

ax_blue = obj_blue.plot(kind='bar', stacked=True, label='Game Outcome', color=['red', 'blue'], alpha=0.8)
ax_blue.set_xlabel('EliteMonsters, Dragons, Heralds')
ax_blue.set_ylabel('Frequency')
ax_blue.set_title('Objective Eliminations : Blue Team')
ax_blue.legend(["Lost Game", "Won Game"], title="Game Outcome")

obj_blue

The stacked plot shown above contains values regarding objective eliminations. As mentioned in section `2.1`, these
objectives include total elite monsters, dragons, & heralds eliminated.

By completing these objectives, a team/player will gain perks that give will give you an advantage such as increased
damage dealing, or regenerating health quicker. The stacked plots above show the clear impact these objectives make on
winning.

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.1.4 Win rate by Kills & GoldPerMinute

In [ ]:
from matplotlib.lines import Line2D

mark1 = {0:"X", 1:"o"}
mark2 = {0:"o", 1:"X"}

legend_elements_1 = [Line2D([0],[0],marker='X',color='w',label='Lost Game',markerfacecolor='red',markersize=8),
                     Line2D([0],[0],marker='o',color='w',label='Won Game',markerfacecolor='blue',markersize=10)]
legend_elements_2 = [Line2D([0],[0],marker='X',color='w',label='Lost Game',markerfacecolor='blue',markersize=8),
                     Line2D([0],[0],marker='o',color='w',label='Won Game',markerfacecolor='red',markersize=10)]

plt.subplots(figsize=(20,5))
sns.set_style("white")

plt.subplot(1,2,1)
ax1 = sns.scatterplot(data=df,x='blueGoldPerMin',y='blueKills',hue='blueWins',palette=['red','blue'],style='blueWins',markers=mark1)
ax1.set_title('Blue Team : Kills & Gold Per Minute')
ax1.set_xlabel('Gold Per Minute')
ax1.set_ylabel('# of Kills')
ax1.legend(title='Game Outcome',handles=legend_elements_1,loc='upper left')

plt.subplot(1,2,2)
ax2 = sns.scatterplot(data=df,x='redGoldPerMin',y='redKills',hue='blueWins',palette=['red','blue'],style='blueWins',markers=mark2)
ax2.set_title('Red Team : Kills & Gold Per Minute')
ax2.set_xlabel('Gold Per Minute')
ax2.set_ylabel('# of Kills')
ax2.legend(title='Game Outcome',handles=legend_elements_2,loc='upper left')

plt.show()

Above we have scatter plots for each teams performances during games that they won/lost.The x-axis
represents the average gold per minute, & the y-axis represents the number of kills. In both scatter
plots we see the same pattern of the higher number a kills & average gold per minute achieved by a
team, will often lead to winning the game.


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.2 Data Relationship Exploration

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.2.1 Correlation Matrix

In [ ]:
cmap = sns.set(style="darkgrid", palette="colorblind")

ax = plt.subplots(figsize=(17,15))
plt.pcolor(df[blue_col].corr())
plt.xticks(np.arange(0, len(blue_col), 1), blue_col)
plt.yticks(np.arange(0, len(blue_col), 1), blue_col)
plt.xticks(rotation=90)
sns.heatmap(df[blue_col].corr(),cmap=cmap,annot=True)
plt.show()

The heatmap printed above contains correlating values pertaining to the 'blueWins' attribute.
We observe that there are 6 attributes with a correlation value of 1 ('blueWins' not included) :
- 'redDeaths'
- 'blueKills'
- 'blueCSPerMin'
- 'blueTotalGold'
- 'blueGoldPerMin'
- 'blueTotalMinionsKilled'


#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.2.2 Winning attributes

In [ ]:
df_win = df[df["blueWins"]==1]
df_lose = df[df["blueWins"]==0]

corr_blue = ['blueKills', 'blueTotalGold', 'blueTotalMinionsKilled', 'blueCSPerMin', 'redDeaths', 'blueGoldPerMin']

c = 0
plt.subplots(figsize=(20,5))
for i in corr_blue:
    plt.subplot(1,3,c+1)
    sns.distplot(df_win[i],label='Won Game',color='blue')
    sns.distplot(df_lose[i],label='Lost Game',color='red')
    plt.legend(title='Game Outcome')
    c=c+1

    if c == 3:
        plt.show()
        if i != corr_blue[len(corr_blue) - 1]:
            plt.subplots(figsize=(20,5))
        c=0

Using the attributes we found in `3.2.1` with a correlation value of 1, we output the distribution plot
(kernel density estimation + histogram) for each attribute to help us visualize how significantly
important each of these are to winning.

The distribution plots shown above help visualize the importance of these attributes and how the higher
these attribute values are, the greater chance a team has at winning the game. Players/teams can adjust
their strategy by using the averaged values displayed in the plots above as a measure of success within
their first 10 minutes of a game.